# 신경망 모델을 사용한 멜로디 시퀀스 학습 

이번 튜토리얼에서는 기존에 데이터로 주어진 곡들과 비슷한 느낌을 주는 멜로디를 생성하는 모델을 만들어보고자 합니다.

멜로디의 생성을 모델링함에 있어서 크게 두가지 어프로치가 있을 수 있습니다. 


1) 첫번째로는 실제 음악 이론과 작곡가의 멜로디 창작 스타일에 대한 분석을 토대로 직접 작곡 규칙을 설계하는 방법입니다.  

2) 두번째로는 악곡 데이터에서 통계적인 분석을 통해 생성 모델을 학습하는 방법입니다. 기본적으로 시퀀셜한 데이터에서 패턴을 찾아내는 작업이기 때문에 다양한 기계학습 방법론이 쓰일 수 있습니다. 


이번 튜토리얼에서는 딥러닝의 토대가 되는 인공신경망 아키텍쳐 중 시퀀스 학습을 위해 만들어진 RNN 구조를 활용해서 접근해보고자 합니다.


## 라이브러리 준비

먼저 필요한 라이브러리를 불러옵니다. 

미디 파일을 다루는 라이브러리로는 여러가지가 있지만 이 튜토리얼에서는 music21을 사용하고자 합니다.

[http://web.mit.edu/music21/](http://web.mit.edu/music21/)

그 외 Numpy, OS단에서의 로컬 path를 핸들링하기 위해 os, 데이터 저장을 위해 pickle을 임포트합니다.


In [1]:
import music21
import numpy as np
import os
import pickle

## 데이터 Preprocessing - Extracting note information from midi file.

우리는 미디 파일에서 음악 정보를 받아온 후 이를 바탕으로 패턴을 학습하는 모델을 만들 예정입니다. 

가장 먼저 필요한 작업은 미디 데이터셋을 준비하는 일입니다.

오픈되어 있는 미디 데이터셋에는 여러가지가 있지만, 단순한 포크송 형태의 기본적인 미디 데이터셋으로 Nottingham Database를 사용하고자 합니다.

1200개의 영국/미국 포크곡으로 이루어져 있는 데이터셋으로, 모든 미디파일은 멜로디, 화성의 두개의 트랙으로 구성되어 있습니다.

또한, 얼마전 음악 AI 스타트업 JukeDeck에서 이 데이터셋을 한번 더 정리(clean up)해서 Github에 공유했기 때문에 저희는 이를 활용하겠습니다.

[https://github.com/jukedeck/nottingham-dataset](https://github.com/jukedeck/nottingham-dataset)

이를 다운 받아서 로컬 하드드라이브의 적당한 위치에 저장합니다.

그런 다음, 데이터가 저장된 폴더를 변수로 지정해 놓습니다.


In [2]:
data_path = '~/Google Drive/data/nottingham-dataset/MIDI/melody/'

먼저 미디 파일 하나를 가지고 테스트를 하기 위해 파일명 하나를 변수로 받아옵니다.

In [3]:
file_name = 'jigs273.mid'

이제 music21 라이브러리에 이미 정의되어 있는 함수를 이용해서 미디 파일 내부의 음악 정보를 해석합니다.

In [4]:
midi_obj = music21.converter.parse(data_path + file_name)

In [5]:
midi_obj

<music21.stream.Score 0x10a01b780>

music21은 라이브러리 내에 정의해놓은 특정한 오브젝트 형태로 정보를 저장합니다. 

이 오브젝트안에는 특정 구조로 음악 정보들이 저장되어 있습니다.

이 구조를 이용해서 우리가 원하는 노트의 음정 / 노트의 시작지점 두가지 정보에 대한 Array를 받아오겠습니다.

In [6]:
def create_mel_data_each_file(midi_obj):
    mel_data = dict()
    
    print(len(midi_obj.flat.getElementsByClass(music21.chord.Chord)))
    for n in midi_obj.flat.getElementsByClass(music21.chord.Chord):
        if n.offset not in mel_data:
            mel_data[n.offset] = dict()
        mel_data[n.offset]['offset'] = n.offset
        for p in n.pitches:
            mel_data[n.offset]['note'] = p.midi
    print(len(midi_obj.flat.getElementsByClass(music21.note.Note)))

    for n in midi_obj.flat.getElementsByClass(music21.note.Note):
        if n.offset not in mel_data:
            mel_data[n.offset] = dict()
        mel_data[n.offset]['offset'] = n.offset
        prev_p = 0
        for p in n.pitches:
            if prev_p < p.midi:
                mel_data[n.offset]['note'] = p.midi
            prev_p = p.midi    
    
    return mel_data

우리가 가지고 있는 데이터에 이미 멜로디만 따로 정리되어 있기 때문에 이 작업이 매우 간단하게 이루어졌습니다.

여러분이 작업하기 원하시는 데이터셋이 있다면 거기서 원하는 정보만 추출하는 작업이 까다로울 수도 있습니다. 보통 어느 트랙에 어떤 악기 파트가 배정되어 있는지도 정리가 되어 있지 않는 경우도 많으니까요.

위 함수에서는 만약 두개 이상의 음이 동시에 발현되는 경우 단순하게 맨 위의 음정만을 멜로디로 치부해서 데이터화하도록 되어있습니다. 

In [7]:
mel_data = create_mel_data_each_file(midi_obj)

0
165


한곡에서 165개의 멜로디 노트 정보를 받아오는 것을 확인했습니다.

자, 이제 우리 데이터셋의 모든 곡에 대해서 노트 정보를 받아와봅시다.

먼저, 폴더안의 모든 미디 파일명을 리스트로 만듭니다.


In [8]:
file_list = []

for file_name in os.listdir(os.path.expanduser(data_path)):
    if file_name.endswith('.mid') or file_name.endswith('.midi'):
        file_list.append(data_path + file_name)

파일 리스트를 확인해봅시다.

In [9]:
file_list 

['~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover1.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover10.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover11.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover12.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover13.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover14.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover15.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover16.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover17.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover18.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover19.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover2.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover20.mid',
 '~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover21.mid',
 '~/Google Drive/data/nottingham-dat

이제 모든 파일들의 path를 받아놓았으니, 루프를 돌면서 모든 파일에 대해서 멜로디 노트 정보를 저장해봅시다.

In [10]:
mel_arr_list = []

for file_name in file_list:
    print(file_name)
    midi_obj = music21.converter.parse(file_name)
    mel_data = create_mel_data_each_file(midi_obj)

    mel_arr = []
    for key, value in sorted(mel_data.items()):
        mel_arr.append(mel_data[key])
    
    mel_arr_list.append(mel_arr)

~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover1.mid
0
68
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover10.mid
0
423
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover11.mid
0
169
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover12.mid
0
239
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover13.mid
0
266
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover14.mid
0
132
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover15.mid
8
173
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover16.mid
0
147
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover17.mid
0
374
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover18.mid
0
180
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover19.mid
68
107
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover2.mid
0
214
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover20.mid
0
305
~/Google Drive/data/nottingham-dataset/MIDI/melody/ashover21.mid
0
208
~/Google

~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs109.mid
0
492
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs11.mid
0
162
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs110.mid
0
2724
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs111.mid
0
502
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs112.mid
0
158
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs113.mid
0
152
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs114.mid
0
170
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs115.mid
0
209
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs116.mid
0
160
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs117.mid
0
230
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs118.mid
0
166
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs119.mid
0
156
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs12.mid
0
112
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs120.mid
0
131
~/Google Drive/data/nottingham-data

0
67
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs218.mid
0
60
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs219.mid
0
66
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs22.mid
0
528
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs220.mid
0
174
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs221.mid
0
276
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs222.mid
0
158
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs223.mid
0
451
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs224.mid
0
156
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs225.mid
0
301
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs226.mid
0
164
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs227.mid
0
292
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs228.mid
0
154
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs229.mid
0
146
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs23.mid
0
150
~/Google Drive/data/nottingham-da

~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs327.mid
0
128
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs328.mid
0
188
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs329.mid
0
416
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs33.mid
0
169
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs330.mid
0
160
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs331.mid
0
158
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs332.mid
0
134
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs333.mid
0
148
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs334.mid
0
160
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs335.mid
0
230
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs336.mid
0
181
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs337.mid
0
246
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs338.mid
0
507
~/Google Drive/data/nottingham-dataset/MIDI/melody/jigs339.mid
0
152
~/Google Drive/data/nottingham-data

0
304
~/Google Drive/data/nottingham-dataset/MIDI/melody/playford3.mid
0
332
~/Google Drive/data/nottingham-dataset/MIDI/melody/playford4.mid
0
182
~/Google Drive/data/nottingham-dataset/MIDI/melody/playford5.mid
0
205
~/Google Drive/data/nottingham-dataset/MIDI/melody/playford6.mid
0
126
~/Google Drive/data/nottingham-dataset/MIDI/melody/playford7.mid
0
144
~/Google Drive/data/nottingham-dataset/MIDI/melody/playford8.mid
0
274
~/Google Drive/data/nottingham-dataset/MIDI/melody/playford9.mid
0
68
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsa-c1.mid
0
120
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsa-c10.mid
0
142
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsa-c11.mid
0
114
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsa-c12.mid
0
226
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsa-c13.mid
0
224
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsa-c14.mid
0
134
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsa-c15.mid
0

0
89
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g34.mid
0
126
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g35.mid
0
110
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g36.mid
0
134
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g37.mid
0
210
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g38.mid
0
177
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g39.mid
0
218
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g4.mid
0
232
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g40.mid
0
156
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g41.mid
42
191
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g42.mid
0
240
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g43.mid
0
132
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g44.mid
0
108
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g45.mid
0
240
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsd-g

0
228
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l68.mid
0
237
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l69.mid
0
236
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l7.mid
0
91
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l70.mid
0
237
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l71.mid
0
236
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l72.mid
0
276
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l73.mid
0
218
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l74.mid
0
144
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l75.mid
0
413
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l76.mid
0
212
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l77.mid
0
242
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l78.mid
0
188
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l79.mid
0
402
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsh-l8

0
286
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t20.mid
0
178
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t21.mid
0
222
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t22.mid
0
230
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t23.mid
0
200
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t24.mid
0
505
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t25.mid
0
132
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t26.mid
0
204
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t27.mid
0
132
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t28.mid
0
208
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t29.mid
0
36
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t3.mid
0
195
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t30.mid
0
184
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t31.mid
0
216
~/Google Drive/data/nottingham-dataset/MIDI/melody/reelsr-t3

0
415
~/Google Drive/data/nottingham-dataset/MIDI/melody/slip6.mid
0
210
~/Google Drive/data/nottingham-dataset/MIDI/melody/slip7.mid
0
336
~/Google Drive/data/nottingham-dataset/MIDI/melody/slip8.mid
0
113
~/Google Drive/data/nottingham-dataset/MIDI/melody/slip9.mid
0
130
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes1.mid
0
186
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes10.mid
0
90
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes11.mid
0
122
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes12.mid
0
101
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes13.mid
0
169
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes14.mid
0
91
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes15.mid
0
192
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes16.mid
0
148
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes17.mid
0
85
~/Google Drive/data/nottingham-dataset/MIDI/melody/waltzes18.mid
0
210
~/Google Drive/data/

전처리해놓은 데이터를 파일로 저장해놓으면 향후에 매번 전처리 작업을 다시 하지 않고 데이터를 불러오기만 할 수 있기 때문에, 적당한 폴더에 파일로 저장해놓겠습니다.

In [11]:
preprocessed_dir = "./preprocessed_data/"
if not os.path.exists(preprocessed_dir):
    os.makedirs(preprocessed_dir)

with open(preprocessed_dir + "mel_arr_list.p", "wb") as fp:   
    pickle.dump(mel_arr_list, fp)

우리가 전처리한 데이터를 한번 확인해봅시다. 

리스트로 만들어 놓았기 때문에 첫번째 곡의(인덱스 0번) 정보를 한번 확인해보겠습니다.

In [12]:
mel_arr_list[0]

[{'note': 76, 'offset': 2.0},
 {'note': 74, 'offset': 3.0},
 {'note': 71, 'offset': 5.0},
 {'note': 69, 'offset': 6.0},
 {'note': 71, 'offset': 7.5},
 {'note': 72, 'offset': 8.0},
 {'note': 71, 'offset': 9.0},
 {'note': 67, 'offset': 11.0},
 {'note': 69, 'offset': 12.0},
 {'note': 76, 'offset': 14.0},
 {'note': 74, 'offset': 15.0},
 {'note': 71, 'offset': 17.0},
 {'note': 69, 'offset': 18.0},
 {'note': 71, 'offset': 19.5},
 {'note': 72, 'offset': 20.0},
 {'note': 71, 'offset': 21.0},
 {'note': 65, 'offset': 22.0},
 {'note': 67, 'offset': 23.0},
 {'note': 76, 'offset': 25.0},
 {'note': 74, 'offset': 26.0},
 {'note': 71, 'offset': 28.0},
 {'note': 69, 'offset': 29.0},
 {'note': 71, 'offset': 30.5},
 {'note': 72, 'offset': 31.0},
 {'note': 71, 'offset': 32.0},
 {'note': 67, 'offset': 34.0},
 {'note': 69, 'offset': 35.0},
 {'note': 76, 'offset': 37.0},
 {'note': 74, 'offset': 38.0},
 {'note': 71, 'offset': 40.0},
 {'note': 69, 'offset': 41.0},
 {'note': 71, 'offset': 42.5},
 {'note': 72, '